Analyse de l'image 

In [7]:
!pip install webcolors==1.3

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0 -> 23.1.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [8]:
exif_data_interested = [
    "DateTimeOriginal",
    "Make",
    "Model",
    "Orientation",
    "Copyright",
    "DateTime",
    "DateTimeDigitized"
]

In [9]:
import os
import json
from PIL import Image, ImageColor
from PIL.ExifTags import TAGS
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import mean_squared_error
import webcolors
import math


def getDominatedColors(imgfile):
    # Parcourir les fichiers dans le répertoire d'images
    print(imgfile)
    numarray = np.array(imgfile.getdata(), np.uint8)
    colors = []
    if len(numarray.shape) == 2:
        clusters = KMeans(n_clusters=2, n_init=2)
        clusters.fit(numarray)
        for i in range(len(clusters.cluster_centers_)):
            colors.append(
                "#%02x%02x%02x"
                % (
                    math.ceil(clusters.cluster_centers_[i][0]),
                    math.ceil(clusters.cluster_centers_[i][1]),
                    math.ceil(clusters.cluster_centers_[i][2]),
                )
            )
    return colors

# https://stackoverflow.com/questions/70967119/webcolors-has-no-attribute-css3-hex-to-names
def get_closest_color(rgb_triplet):
    min_colours = {}
    for key, name in webcolors.css3_hex_to_names.items():
        r_c, g_c, b_c = webcolors.hex_to_rgb(key)
        rd = (r_c - rgb_triplet[0]) ** 2
        gd = (g_c - rgb_triplet[1]) ** 2
        bd = (b_c - rgb_triplet[2]) ** 2
        min_colours[(rd + gd + bd)] = name
    return min_colours[min(min_colours.keys())]

class CustomEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, bytes):
            return obj.decode('utf-8', 'ignore')
        return super().default(obj)

metadata = []

# Parcourir les fichiers dans le répertoire d'images
for filename in os.listdir("images"):
    if filename.endswith('.jpg') or filename.endswith('.JPG') or filename.endswith('.jpeg') or filename.endswith('.png'):
        with Image.open(os.path.join("images", filename)) as img:
            json_path = os.path.join("metadata", filename.replace(".jpg", ".json").replace(".JPG", ".json").replace(".jpeg", ".json").replace(".png", ".json"))
            if os.path.exists(json_path):
                with open(json_path, 'r') as f:
                    existing_metadata = json.load(f)
            # else:
            #     existing_metadata = []

            if 'dominated_colors_name' in existing_metadata.keys() :
                print("Already treated")
                pass

            exif_data = img.getexif()
            exif = {}
            if exif_data:
                for tag, value in exif_data.items():
                    decoded_tag = TAGS.get(tag, tag)
                    if decoded_tag in exif_data_interested:
                        exif[decoded_tag] = value

            colors_dominated = getDominatedColors(img)
            colors_dominated_name = []

            print(colors_dominated)

            for color in colors_dominated:
                colors_dominated_name.append(get_closest_color(ImageColor.getcolor(color, "RGB")))



            # Add the new metadata to the list
            # print(existing_metadata)

            existing_metadata['filename']= filename
            existing_metadata['width']= img.width
            existing_metadata['height']= img.height
            existing_metadata['format']= img.format
            existing_metadata['mode']= img.mode
            existing_metadata['dominated_colors']= colors_dominated
            existing_metadata['dominated_colors_name']= colors_dominated_name
            existing_metadata['exif']= exif


            # Save the metadata to a JSON file
            with open(json_path, 'w') as f:
                json.dump(existing_metadata, f, cls=CustomEncoder)

<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=4504x3378 at 0x7F3F72248CA0>
['#9d9d62', '#4f4d2a']
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=3573x3573 at 0x7F3FAC655610>
['#97b087', '#4b5332']
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=2738x2104 at 0x7F3FAC6556A0>
['#b7b2a4', '#56533b']
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=3072x2048 at 0x7F3FAC655520>
['#5f4f31', '#b8ad9e']
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=756x1008 at 0x7F3FAC655640>
['#384315', '#b0b65e']
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=2304x3072 at 0x7F3FAC655820>
['#375d25', '#9aad7c']
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=2544x1696 at 0x7F3FAC6556A0>
['#a8c385', '#536132']
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=2717x3302 at 0x7F3FAC6554F0>
['#b4cc4b', '#343f28']
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=2593x1859 at 0x7F3FAC6550A0>
['#46512a', '#cfc9a3']
<PIL.JpegImagePlugin.JpegImageFile ima